In [1]:
import pathlib
from itertools import count

import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

from qec_util import Layout
from qec_util.layouts import plot as plot_layout

from surface_sim import Setup
from surface_sim.models import CircuitNoiseModel
from surface_sim.experiments import memory_exp

In [14]:
def sample_outcomes(experiment, seed, num_shots):
    sampler = experiment.compile_sampler(seed=seed)
    outcomes = sampler.sample(num_shots)
    return outcomes

def outcome_dataset(outcomes, num_shots, num_rounds, num_anc):
    outcomes = outcomes.reshape(num_shots, -1)
    anc_outcomes, data_outcomes = np.split(
        outcomes, [num_rounds * num_anc], axis=1
    )
    anc_outcomes = anc_outcomes.reshape(num_shots, num_rounds, num_anc)

    anc_meas = xr.DataArray(
        data=anc_outcomes.astype(int),
        dims=["shot", "qec_round", "anc_qubit"],
    )

    data_meas = xr.DataArray(
        data=data_outcomes.astype(int),
        dims=["shot", "data_qubit"],
    )

    dataset = xr.Dataset(
        data_vars=dict(
            anc_meas=anc_meas,
            data_meas=data_meas,
        )
    )

    return dataset

In [12]:
EXP_NAME = "d3_rot-surf_circ-level_example"

NOTEBOOK_DIR = pathlib.Path.cwd()

LAYOUT_DIR = NOTEBOOK_DIR / "layouts"
if not LAYOUT_DIR.exists():
    raise ValueError("Layout directory does not exist.")

SETUP_DIR = NOTEBOOK_DIR / "setups"
if not SETUP_DIR.exists():
    raise ValueError("Setup directory does not exist.")

DATA_DIR = NOTEBOOK_DIR / "data" / EXP_NAME
DATASET_TYPES = ("train", "dev", "test")

In [3]:
LAYOUT_FILE = "d3_rotated_layout.yaml"
layout = Layout.from_yaml(LAYOUT_DIR / LAYOUT_FILE)

SETUP_FILE = "circuit_level_noise.yaml"
setup = Setup.from_yaml(SETUP_DIR / SETUP_FILE)

model = CircuitNoiseModel(setup, layout)

# Generate the training data

In [10]:
DATASET_TYPE = "train"

SEED = 4028 # Initial seed for the RNG
LOG_STATES = [0, 1] # Logical state
NUM_ROUNDS = 60 # Number of rounds
NUM_SHOTS = 10000 # Number of shots
ROT_BASIS = False  # In the z-basis
MEASUREMENT_RESET = False # No resets following measurements

In [15]:
assert DATASET_TYPE in DATASET_TYPES

qec_rounds = list(range(1, NUM_ROUNDS + 1))
shots = list(range(1, NUM_SHOTS + 1))

data_qubits = layout.get_qubits(role="data")
anc_qubits = layout.get_qubits(role="anc")
dist = layout.distance

num_anc = len(anc_qubits)
rng = np.random.default_rng(SEED)

datasets = []

for log_state in LOG_STATES:
    seed = rng.integers(0, 4294967296)

    basis = "X" if ROT_BASIS else "Z"
    exp_name = f"surf-code_d{dist}_b{basis}_s{log_state}_n{NUM_SHOTS}_r{NUM_ROUNDS}"

    exp_folder = DATA_DIR / DATASET_TYPE / exp_name
    exp_folder.mkdir(parents=True, exist_ok=True)

    experiment = memory_exp(
        model,
        num_rounds=NUM_ROUNDS,
        log_state=log_state,
        rot_basis=ROT_BASIS,
        meas_reset=MEASUREMENT_RESET,
    )

    experiment.to_file(exp_folder / "circuit")

    outcomes = sample_outcomes(experiment, seed, NUM_SHOTS)
    dataset = outcome_dataset(outcomes, NUM_SHOTS, NUM_ROUNDS, num_anc)

    dataset = dataset.assign_coords(
        seed=seed,
        shot=shots,
        qec_round=qec_rounds,
        data_qubit=data_qubits,
        anc_qubits=anc_qubits,
        log_state=log_state,
        rot_basis=int(ROT_BASIS),
        meas_reset=int(MEASUREMENT_RESET),
    )

    dataset.to_netcdf(exp_folder / "measurements.nc")

    error_model = experiment.detector_error_model(
        decompose_errors=True,
        allow_gauge_detectors=True,
    )
    error_model.to_file(exp_folder / "detector_error_model")
